In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn import *
from catboost import CatBoostRegressor
import warnings as w
w.filterwarnings('ignore')

import matplotlib.pyplot as plt
%matplotlib inline

C:\Users\PC\anaconda3\envs\yeseul\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\PC\anaconda3\envs\yeseul\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\PC\anaconda3\envs\yeseul\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
answer = pd.read_csv("../data/4.체감안전도_통계(2017~2019).csv")

FileNotFoundError: [Errno 2] No such file or directory: '../data/4.체감안전도_통계(2017~2019).csv'

In [ ]:
sub = pd.read_csv("../data/25.결과제출양식.csv")
pol_order = list(sub['경찰서명'])

for i in range(len(pol_order)):
    pol_order[i] = "jur_stn_"+pol_order[i]
    pol_order[i] = pol_order[i].replace(" ","")

In [ ]:
tqq = pd.read_csv("q1_train.csv")#,index_col=0)

In [ ]:
train_data = pd.read_csv("q1_train.csv")#,index_col=0)
target_data = pd.read_csv("q1_target.csv")#,index_col=0)

target_data.reset_index(drop=True,inplace=True)

In [ ]:
train_data = pd.get_dummies(train_data)
target_data = pd.get_dummies(target_data)

In [ ]:
train_data.loc[151,'cctv_cnt'] = 2527.0
train_data.loc[151,'bell_us'] = 200
train_data.loc[151,'light_cnt'] = 10410.0

In [ ]:
target_data['상반기'] =target_data['date_2020(상반기)']
del target_data['date_2020(상반기)'],target_data['date_2020(하반기)']

train_data['상반기'] = ( train_data['date_2018(상반기)'] | train_data['date_2019(상반기)'] )
del train_data['date_2018(상반기)'],train_data['date_2018(하반기)'],train_data['date_2019(상반기)'],train_data['date_2019(하반기)']

In [ ]:
for i in pol_order:
    del train_data[i]

In [ ]:
tar_sang = target_data[target_data['상반기']==1]
tar_ha = target_data[target_data['상반기']!=1]

tar_sang.reset_index(drop=True,inplace=True)
tar_ha.reset_index(drop=True,inplace=True)

# del tar_sang['jur_stn']
# del tar_sang['date']
# del tar_ha['jur_stn']
# del tar_ha['date']

In [ ]:
for i in pol_order:
    del tar_sang[i],tar_ha[i]

In [ ]:
tar_sang.columns

In [ ]:
y = train_data['rob_safety_q1']
X = train_data
del X['rob_safety_q1']

<hr>

In [ ]:
model = CatBoostRegressor(iterations=2500,silent=True,learning_rate=0.005072).fit(X, y)

In [ ]:
impotances = model.feature_importances_
feature_list = list(zip(model.feature_importances_,X.columns))
feature_list.sort(reverse=True)
feature_list=feature_list[:20]
feature_list.sort(reverse=False)
feature_df = pd.DataFrame(feature_list,columns=["score","feature"])
plt.figure(figsize=(8,6),facecolor="#F0FDFF")
plt.title("Feature_Importances : {}".format(str(model).split('.')[0].split('<')[-1]), fontsize="xx-large")
plt.barh(feature_df["feature"],feature_df["score"],color="#C00000")
plt.grid(color="#CFE8E1")

In [ ]:
df = pd.DataFrame(list(zip(model.feature_importances_,X.columns)))
df.columns = ['weight','columns']
df.sort_values(['weight'],axis=0,ascending=False)

In [ ]:
# col_num = 23
# row=103
# print(tqq.loc[row,'jur_stn'])
# tester = (X.loc[row,:]).copy()
# target_col_value = X.columns[col_num]
# print("- 타겟 피쳐 : ",target_col_value,"\n- 값 : ", tester[target_col_value])
# print("- 기존 만족도 : ",y[row])
# print("- Feature Importance : ",model.feature_importances_[col_num])
# target_var_value = 1.1
# tester[target_col_value] = tester[target_col_value] * target_var_value
# print("- 수치 10% 증가 시 예측 만족도 : ",model.predict(np.array(tester)))
# print("* 변화량 : ",(model.predict(np.array(tester))-y[row]))

In [ ]:
pred = (model.predict(tar_ha)* 0.85525 + model.predict(tar_sang) * 0.15525)

In [ ]:
prediction = pd.Series(pred)

In [ ]:
prediction.to_csv("q1_result.csv")